In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [2]:
#function for easier splitting string with multiple delimeters
def split(delimiters, string, maxsplit=0):
    regexPattern = '|'.join(map(re.escape, delimiters))
    return re.split(regexPattern, string, maxsplit)

In [3]:
link  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [4]:
#create BeautifulSoup object for parsing
url = requests.get(link)
htmltext = url.text
soup = BeautifulSoup(htmltext, 'html.parser')

In [5]:
#select table in html
table = soup.select('table[rules="all"] p')

In [6]:
#creating dataframe for future data inserting
tabler = pd.DataFrame(columns=["PostalCode","Borough","Neighborhood"])

In [7]:

postal = []
borough = []
neighborhood =[]
#iterate all of table elements
for i in table:
    currentPostal = i.find("b").text
    text = i.find("span").text
    if text == "Not assigned":
        continue
    text = split(["(",")","/"],text)
    currentBorough = text[0]
    #iterate all neighborhood but without borough
    for i in text[1:]:
        if i != " " and i!= '':
            if text.index(i) == 1:
                neighborhood_s  = i
            else:
                neighborhood_s=neighborhood_s+ ", "+i.strip()
    #insering data to lists
    postal.append(currentPostal)
    borough.append(currentBorough)
    neighborhood.append(neighborhood_s)

In [10]:
#inserting data to dataframe from lists
tabler = pd.DataFrame(list(zip(postal,borough,neighborhood)),columns=["PostalCode","Borough","Neighborhood"])
tabler.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [11]:
tabler.shape

(103, 3)

In [12]:
#get data about coordinates from file
data = pd.read_csv("Geospatial_Coordinates.csv")
data = data.rename(columns={"Postal Code":"PostalCode"})
#merging two tables by postal code
merging = pd.merge(tabler,data,on="PostalCode")
merging.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
